# Project Title
### Data Engineering Capstone Project

#### Project Summary
ETL using PySpark using US-Immigration Data.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1678084306076_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Step 1: Scope the Project and Gather Data

#### Scope 
After investiagting as sample of the Data, and creating the Conceptual model, I loaded the full data sets into S3 in same region of this EMR o implement this model using PySpark.

#### Describe and Gather Data 
I'm using US-Immigration data, and US Cities Demographics

## 1. LOADING IMMIGRATION DATA

In [2]:
df_immigration=spark.read.parquet("s3://aws-logs-557613354735-us-east-1/New folder/sas_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_immigration.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3096313

In [4]:
df_immigration.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [5]:
df_i94 = df_immigration.select([
    "cicid", "i94yr", "i94mon", "i94cit", "i94res", "i94mode", "i94bir",
    "i94visa", "gender", "arrdate", "i94addr"
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_i94

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[cicid: double, i94yr: double, i94mon: double, i94cit: double, i94res: double, i94mode: double, i94bir: double, i94visa: double, gender: string, arrdate: double, i94addr: string]

In [7]:
df_city_dem = spark.read.option("delimiter", ";").csv("s3://aws-logs-557613354735-us-east-1/New folder/us-cities-demographics.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_city_dem.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+--------------------+----------+------------------+-----+
|          _c0|          _c1|       _c2|            _c3|              _c4|             _c5|               _c6|         _c7|                 _c8|       _c9|              _c10| _c11|
+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+--------------------+----------+------------------+-----+
|         City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household...|State Code|              Race|Count|
|Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                 2.6|        MD|Hispanic or Latino|25924|
|       Quincy|Massachusetts|      41.0|          44129|            49500|           93629|    

In [9]:
df_mode  = spark.read.csv("s3://aws-logs-557613354735-us-east-1/New folder/data_n/i94mode.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_mode.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+
| _c0| _c1|
+----+----+
|code|mode|
|   1| Air|
+----+----+
only showing top 2 rows

In [11]:
df_country = spark.read.csv("s3://aws-logs-557613354735-us-east-1/New folder/data_n/i94cit_res.csv")
df_visa = spark.read.csv("s3://aws-logs-557613354735-us-east-1/New folder/data_n/i94visa.csv")
df_address = spark.read.csv("s3://aws-logs-557613354735-us-east-1/New folder/data_n/i94addr.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_country.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
|code|             country|
| 525|BRITISH VIRGIN IS...|
+----+--------------------+
only showing top 2 rows

In [13]:
df_visa.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
| _c0|      _c1|
+----+---------+
|code|visa_type|
|   1| Business|
+----+---------+
only showing top 2 rows

In [14]:
df_address.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+
| _c0|    _c1|
+----+-------+
|code|  state|
|  AL|ALABAMA|
+----+-------+
only showing top 2 rows

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
- Immigratinon Data
    1. Choose only the wanted columns.
    2. Fixing Data Types
    3. Get Date from SAS Date
    4. Rename Columns
    5. drop negative ages
- OTHER
   1. fix headers

In [15]:
df_i94 \
    .groupby(['cicid']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
|cicid|count|
+-----+-----+
+-----+-----+

In [16]:
df_i94.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94addr: string (nullable = true)

In [17]:
from pyspark.sql.types import IntegerType, DateType
df_i94 = df_i94.withColumn("cicid",df_i94.cicid.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94yr",df_i94.i94yr.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94mon",df_i94.i94mon.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94cit",df_i94.i94cit.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94res",df_i94.i94res.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94mode",df_i94.i94mode.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94bir",df_i94.i94bir.cast(IntegerType()))
df_i94 = df_i94.withColumn("i94visa",df_i94.i94visa.cast(IntegerType()))
df_i94 = df_i94.withColumn("arrdate",df_i94.arrdate.cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_i94.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94addr: string (nullable = true)

In [19]:
from pyspark.sql import functions as F
from datetime import datetime, timedelta
def get_date(x):
    """
    Getting Dates from SAD DATE
    # https://libguides.library.kent.edu/SAS/DatesTime
    """
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=x)
    except:
        return None
    
get_date = F.udf(get_date, DateType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_i94.select('arrdate', get_date('arrdate').alias("date")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
|arrdate|      date|
+-------+----------+
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
|  20574|2016-04-30|
+-------+----------+
only showing top 20 rows

In [21]:
df_i94 = df_i94.withColumn("arrdate", get_date(df_i94.arrdate)) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_i94.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- i94addr: string (nullable = true)

In [23]:
df_i94 = df_i94.withColumnRenamed("cicid", "id")
df_i94 = df_i94.withColumnRenamed("i94yr", "year")
df_i94 = df_i94.withColumnRenamed("i94mon", "month")
df_i94 = df_i94.withColumnRenamed("i94cit", "citizenship_code")
df_i94 = df_i94.withColumnRenamed("i94res", "residence_code")
df_i94 = df_i94.withColumnRenamed("i94mode", "mode")
df_i94 = df_i94.withColumnRenamed("i94bir", "age")
df_i94 = df_i94.withColumnRenamed("i94visa", "visa_type")
df_i94 = df_i94.withColumnRenamed("arrdate", "dt_arrival")
df_i94 = df_i94.withColumnRenamed("i94addr", "address")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_i94.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
|     id|year|month|citizenship_code|residence_code|mode|age|visa_type|gender|dt_arrival|address|
+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
|5748517|2016|    4|             245|           438|   1| 40|        1|     F|2016-04-30|     CA|
|5748518|2016|    4|             245|           438|   1| 32|        1|     F|2016-04-30|     NV|
+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
only showing top 2 rows

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49254)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(req

In [25]:
df_i94.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- citizenship_code: integer (nullable = true)
 |-- residence_code: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_type: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- dt_arrival: date (nullable = true)
 |-- address: string (nullable = true)

In [26]:
df_i94.createOrReplaceTempView('i94')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
spark.sql("SELECT * FROM i94 LIMIT 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
|     id|year|month|citizenship_code|residence_code|mode|age|visa_type|gender|dt_arrival|address|
+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
|1817663|2016|    4|             148|           131|   1| 23|        2|     F|2016-04-10|     CA|
|1817664|2016|    4|             148|           131|   1| 29|        1|     M|2016-04-10|     CA|
|1817665|2016|    4|             148|           131|   1| 47|        1|     M|2016-04-10|     CA|
|1817666|2016|    4|             148|           131|   1| 37|        2|     F|2016-04-10|     CA|
|1817667|2016|    4|             148|           131|   1| 41|        2|     M|2016-04-10|     CA|
+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+

In [28]:
spark.sql("SELECT max(dt_arrival), min(dt_arrival) FROM i94 LIMIT 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------------+
|max(dt_arrival)|min(dt_arrival)|
+---------------+---------------+
|     2016-04-30|     2016-04-01|
+---------------+---------------+

In [29]:
spark.sql("SELECT max(age), min(age), avg(age) FROM i94 LIMIT 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+------------------+
|max(age)|min(age)|          avg(age)|
+--------+--------+------------------+
|     114|      -3|41.767614458485205|
+--------+--------+------------------+

## drop -ve ages

In [30]:
df_i94 = df_i94.where(df_i94.age > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_i94.createOrReplaceTempView('i94')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
spark.sql("SELECT max(age), min(age), avg(age) FROM i94 LIMIT 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+-----------------+
|max(age)|min(age)|         avg(age)|
+--------+--------+-----------------+
|     114|       1|41.77795359553049|
+--------+--------+-----------------+

In [33]:
df_city_dem.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(_c0='City', _c1='State', _c2='Median Age', _c3='Male Population', _c4='Female Population', _c5='Total Population', _c6='Number of Veterans', _c7='Foreign-born', _c8='Average Household Size', _c9='State Code', _c10='Race', _c11='Count')]

In [34]:
r = """c0='City', _c1='State', _c2='Median Age', _c3='Male Population', _c4='Female Population', _c5='Total Population', _c6='Number of Veterans', _c7='Foreign-born', _c8='Average Household Size', _c9='State Code', _c10='Race', _c11='Count'"""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
for i in r.split(','):
    old, new = i.replace("'","").split("=")
    df_city_dem = df_city_dem.withColumnRenamed(old.strip(), new.strip())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_city_dem.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|             _c0|         State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race| Count|
+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|            City|         State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|  Average Household...|State Code|                Race| Count|
|   Silver Spring|      Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino| 25924|
|          Quincy| Massachusetts|      41.0|      

In [37]:
df_city_dem = df_city_dem.withColumnRenamed("_c0", "City")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_city_dem.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|   State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|              Race|Count|
+-------------+--------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|   State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|  Average Household...|State Code|              Race|Count|
|Silver Spring|Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|Hispanic or Latino|25924|
+-------------+--------+----------+---------------+-----------------+----------------+------------------+-----

In [39]:
# remove first row
df_city_dem = df_city_dem.filter(df_city_dem.City != "City")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_city_dem.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|              Race|Count|
+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|Hispanic or Latino|25924|
|       Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|             White|58723|
+-------------+-------------+----------+---------------+-----------------+-----------

In [41]:
df_city_dem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)

In [42]:
df_city_dem = df_city_dem.withColumnRenamed("City", "city")
df_city_dem = df_city_dem.withColumnRenamed("State", "state")
df_city_dem = df_city_dem.withColumnRenamed("Median Age", "median_age")
df_city_dem = df_city_dem.withColumnRenamed("Male Population", "male_population")
df_city_dem = df_city_dem.withColumnRenamed("Female Population", "female_population")
df_city_dem = df_city_dem.withColumnRenamed("Total Population", "total_population")
df_city_dem = df_city_dem.withColumnRenamed("Number of Veterans", "n_veterans")
df_city_dem = df_city_dem.withColumnRenamed("Foreign-born", "foreign_born")
df_city_dem = df_city_dem.withColumnRenamed("Average Household Size", "avg_household_size")
df_city_dem = df_city_dem.withColumnRenamed("State Code", "state_code")
df_city_dem = df_city_dem.withColumnRenamed("Race", "race")
df_city_dem = df_city_dem.withColumnRenamed("City", "city")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df_city_dem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: string (nullable = true)
 |-- male_population: string (nullable = true)
 |-- female_population: string (nullable = true)
 |-- total_population: string (nullable = true)
 |-- n_veterans: string (nullable = true)
 |-- foreign_born: string (nullable = true)
 |-- avg_household_size: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- Count: string (nullable = true)

In [44]:
df_city_dem = df_city_dem.withColumn("n_veterans",df_city_dem.n_veterans.cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df_city_dem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: string (nullable = true)
 |-- male_population: string (nullable = true)
 |-- female_population: string (nullable = true)
 |-- total_population: string (nullable = true)
 |-- n_veterans: integer (nullable = true)
 |-- foreign_born: string (nullable = true)
 |-- avg_household_size: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- Count: string (nullable = true)

In [46]:
from pyspark.sql.types import DoubleType

df_city_dem = df_city_dem.withColumn("Count",df_city_dem.Count.cast(IntegerType()))
df_city_dem = df_city_dem.withColumn("state_code",df_city_dem.state_code.cast(IntegerType()))
df_city_dem = df_city_dem.withColumn("foreign_born",df_city_dem.foreign_born.cast(IntegerType()))
df_city_dem = df_city_dem.withColumn("avg_household_size",df_city_dem.avg_household_size.cast(DoubleType()))
df_city_dem = df_city_dem.withColumn("total_population",df_city_dem.total_population.cast(DoubleType()))
df_city_dem = df_city_dem.withColumn("female_population",df_city_dem.female_population.cast(DoubleType()))
df_city_dem = df_city_dem.withColumn("male_population",df_city_dem.male_population.cast(DoubleType()))
df_city_dem = df_city_dem.withColumn("median_age",df_city_dem.median_age.cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df_city_dem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: double (nullable = true)
 |-- male_population: double (nullable = true)
 |-- female_population: double (nullable = true)
 |-- total_population: double (nullable = true)
 |-- n_veterans: integer (nullable = true)
 |-- foreign_born: integer (nullable = true)
 |-- avg_household_size: double (nullable = true)
 |-- state_code: integer (nullable = true)
 |-- race: string (nullable = true)
 |-- Count: integer (nullable = true)

In [86]:
df_city_dem.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+----------+---------------+-----------------+----------------+----------+------------+------------------+----------+------------------+-----+
|         city|        state|median_age|male_population|female_population|total_population|n_veterans|foreign_born|avg_household_size|state_code|              race|Count|
+-------------+-------------+----------+---------------+-----------------+----------------+----------+------------+------------------+----------+------------------+-----+
|Silver Spring|     Maryland|      33.8|        40601.0|          41862.0|         82463.0|      1562|       30908|               2.6|      null|Hispanic or Latino|25924|
|       Quincy|Massachusetts|      41.0|        44129.0|          49500.0|         93629.0|      4147|       32935|              2.39|      null|             White|58723|
|       Hoover|      Alabama|      38.5|        38040.0|          46799.0|         84839.0|      4819|        8229|              2.58|      null|

In [49]:
df_city_dem.select(["city",  "median_age", "male_population"]).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+---------------+
|            city|median_age|male_population|
+----------------+----------+---------------+
|   Silver Spring|      33.8|        40601.0|
|          Quincy|      41.0|        44129.0|
|          Hoover|      38.5|        38040.0|
|Rancho Cucamonga|      34.5|        88127.0|
|          Newark|      34.6|       138040.0|
+----------------+----------+---------------+
only showing top 5 rows

In [50]:
df_mode.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+
| _c0| _c1|
+----+----+
|code|mode|
|   1| Air|
+----+----+
only showing top 2 rows

In [51]:
df_mode = df_mode.withColumnRenamed("_c0", "code")
df_mode = df_mode.withColumnRenamed("_c1", "mode")
df_mode = df_mode.filter(df_mode.code != "code")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df_mode.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+
|code|mode|
+----+----+
|   1| Air|
|   2| Sea|
+----+----+
only showing top 2 rows

In [53]:
df_mode = df_mode.withColumnRenamed("_c0", "code")
df_mode = df_mode.withColumnRenamed("_c1", "mode")
df_mode = df_mode.filter(df_mode.code != "code")
df_mode = df_mode.withColumn("code",df_mode.code.cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df_mode.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- code: integer (nullable = true)
 |-- mode: string (nullable = true)

In [55]:
df_country.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
|code|             country|
| 525|BRITISH VIRGIN IS...|
+----+--------------------+
only showing top 2 rows

In [56]:
df_country = df_country.withColumnRenamed("_c0", "code")
df_country = df_country.withColumnRenamed("_c1", "country")
df_country = df_country.filter(df_country.code != "code")
df_country = df_country.withColumn("code",df_country.code.cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
df_country.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+
|code|             country|
+----+--------------------+
| 525|BRITISH VIRGIN IS...|
| 217|              BRUNEI|
+----+--------------------+
only showing top 2 rows

In [58]:
df_country.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- code: integer (nullable = true)
 |-- country: string (nullable = true)

In [59]:
df_visa.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
| _c0|      _c1|
+----+---------+
|code|visa_type|
|   1| Business|
+----+---------+
only showing top 2 rows

In [60]:
df_visa = df_visa.withColumnRenamed("_c0", "code")
df_visa = df_visa.withColumnRenamed("_c1", "visa_type")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
df_visa = df_visa.withColumn("code",df_visa.code.cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
df_visa.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
|code|visa_type|
+----+---------+
|null|visa_type|
|   1| Business|
+----+---------+
only showing top 2 rows

In [63]:
df_visa = df_visa.filter(df_visa.visa_type != 'visa_type')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_visa.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
|code|visa_type|
+----+---------+
|   1| Business|
|   2| Pleasure|
+----+---------+
only showing top 2 rows

In [65]:
df_visa.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- code: integer (nullable = true)
 |-- visa_type: string (nullable = true)

In [66]:
df_address.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+
| _c0|    _c1|
+----+-------+
|code|  state|
|  AL|ALABAMA|
+----+-------+
only showing top 2 rows

In [67]:
df_address = df_address.withColumnRenamed("_c0", "code")
df_address = df_address.withColumnRenamed("_c1", "state")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
df_address = df_address.filter(df_address.state != 'state')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df_address = df_address.withColumn("code",df_address.code.cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
df_address.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+
|code|  state|
+----+-------+
|null|ALABAMA|
|null| ALASKA|
+----+-------+
only showing top 2 rows

In [71]:
df_address.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- code: integer (nullable = true)
 |-- state: string (nullable = true)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
![mode](https://github.com/adelabuhashim/ETL/blob/master/model.png?raw=true)
#### 3.2 Mapping Out Data Pipelines
in the wrangling step I got the tables I want to ingest the new tables schame

In [72]:
df_i94.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
|     id|year|month|citizenship_code|residence_code|mode|age|visa_type|gender|dt_arrival|address|
+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
|5748517|2016|    4|             245|           438|   1| 40|        1|     F|2016-04-30|     CA|
|5748518|2016|    4|             245|           438|   1| 32|        1|     F|2016-04-30|     NV|
+-------+----+-----+----------------+--------------+----+---+---------+------+----------+-------+
only showing top 2 rows

In [73]:
import os
df_i94.write.partitionBy("address").mode('overwrite')\
    .parquet('i94')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
df_mode.write.mode('overwrite').parquet('mode_code')
df_address.write.mode('overwrite').parquet('state_code')
df_visa.write.mode('overwrite').parquet('visa_type_code')
df_country.write.mode('overwrite').parquet('country_code')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df_city_dem.write.mode('overwrite').parquet('city_demographics')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Creating arrival_time table

In [76]:
df_i94.select("dt_arrival").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|dt_arrival|
+----------+
|2016-04-30|
|2016-04-30|
|2016-04-30|
|2016-04-30|
|2016-04-30|
+----------+
only showing top 5 rows

In [77]:
df_i94.select("dt_arrival").createOrReplaceTempView('d')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
spark.sql("SELECT * FROM d LIMIT 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|dt_arrival|
+----------+
|2016-04-30|
|2016-04-30|
|2016-04-30|
|2016-04-30|
|2016-04-30|
+----------+

In [79]:
spark.sql("""SELECT year(dt_arrival) as year, month(dt_arrival) AS month, day(dt_arrival) AS day, weekday(dt_arrival) AS weekday
FROM d""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+-------+
|year|month|day|weekday|
+----+-----+---+-------+
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
|2016|    4| 30|      5|
+----+-----+---+-------+
only showing top 20 rows

In [80]:
spark.sql("""SELECT dt_arrival as time,
  year(dt_arrival) as year, 
  month(dt_arrival) AS month, 
  day(dt_arrival) AS day, 
  weekday(dt_arrival) AS weekday 
FROM 
  d
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+---+-------+
|      time|year|month|day|weekday|
+----------+----+-----+---+-------+
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
|2016-04-30|2016|    4| 30|      5|
+----------+----+-----+---+-------+
only showing top 20 rows

In [81]:
df_times = spark.sql("""SELECT dt_arrival as time,
  year(dt_arrival) as year, 
  month(dt_arrival) AS month, 
  day(dt_arrival) AS day, 
  weekday(dt_arrival) AS weekday 
FROM 
  d
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
df_times = df_times.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
df_times.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

30

In [84]:
df_times.write.mode('overwrite').parquet('arrival_time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
df_times.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- time: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: integer (nullable = true)

## Data Quality Checks

In [89]:
def check_quality_check(df):
    """
    Check if the the data complete or not
    """
    if (df.count() == 0):
        print("Data quality check failed  with zero records!")
    else:
        print("Data quality check passed with record_count: {} records.".format(df.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
check_quality_check(df_i94)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data quality check passed with record_count: 3094745 records.

In [91]:
check_quality_check(df_mode)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data quality check passed with record_count: 4 records.

In [92]:
check_quality_check(df_visa)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data quality check passed with record_count: 3 records.

In [93]:
check_quality_check(df_address)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data quality check passed with record_count: 55 records.

In [95]:
check_quality_check(df_city_dem)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data quality check passed with record_count: 2891 records.

#### 4 Data dictionary 


#### Step 5: Complete Project Write Up
* I choosed Pyspark installed over EMR within AWS Cluster, why?
    1. No one has cluster on his home.
    2. It requires a lot of effort to install pyspark in local mode.
    3. it is easier to connectwith the data on AWS S3.
Before going into the cloud I did soma wrangling using pandas as it is easy and fast using small dataset, also I'm expert on it :D

* The data may be updated for each bath, we can say monthly batchs for evey persiod of the application..
* Write a description of how you would approach the problem differently under the following scenarios:
 * If the data becam 100X large it is no prblem as were using Spark on large cluster adn we can store it for hundreds of those data.
 * We can connect with tableu with batch streaming to load the data in this time daily to easy load and reload in using (better performance).
 * We can make usr groups that can connect ot the same data base or we can easily meake the s3 saved mode pulic.
 * For Data Qullaity I checked the number of rows in each table bfore saveing them as parquest